In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import sys
module_path = os.path.abspath(os.path.join('../../../../../../AgentTorch/'))
if module_path not in sys.path:
    sys.path.append(module_path)

from AgentTorch.helpers import discrete_sample, logical_and, logical_not
from AgentTorch.substep import SubstepTransition

/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/torch_geometric/typing.py:18: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: dlopen(/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/libpyg.so, 6): Library not loaded: /usr/local/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/Python
  Referenced from: /Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/libpyg.so
  Reason: image not found
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/torch_geometric/typing.py:42: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: dlopen(/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/libpyg.so, 6): Library not loaded: /usr/local/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/Python
  

In [2]:
num_agents = 5
num_steps = 10
SUSCEPTIBLE_VAR = 0
RECOVERED_VAR = 3
INFINITY_TIME = 2*num_steps

AWAITING_RESULT_VAR = 1
GOT_RESULT_VAR = -1

In [3]:
def get_tested(t, agents_result_dates, current_stages, is_quarantined, agents_awaiting_test_results, days_to_result, test_compliance_prob):
    
    not_susceptible = (current_stages > SUSCEPTIBLE_VAR).long()
    not_recovered = (current_stages < RECOVERED_VAR).long()
    exposed_infected_agents = logical_and(not_susceptible, not_recovered)

    test_eligible = logical_and(exposed_infected_agents, logical_not(is_quarantined))
    test_eligible = logical_and(test_eligible, logical_not(agents_awaiting_test_results))

    # update awaiting test result
    test_complying = discrete_sample(sample_prob=test_compliance_prob, size=(num_agents,), device='cpu')
    test_enrolled = logical_and(test_eligible, test_complying)
    print("test eligible and enrolled: ", test_eligible, test_enrolled)
    agents_awaiting_test_results = agents_awaiting_test_results + test_enrolled*AWAITING_RESULT_VAR

    # update test result date: sample test result date and assign it to individual agents
    #agents_result_dates = agents_result_dates + test_enrolled*(t + days_to_result - INFINITY_TIME)
    agents_result_dates[test_enrolled.bool()] = (t + days_to_result)

    return agents_awaiting_test_results, agents_result_dates

In [6]:
def get_test_result(t, agents_awaiting_results, agent_result_dates, current_stages, true_positive_prob, false_positive_prob):
    '''Agents receive test result'''

    agents_result_expected_today = (agent_result_dates == t).long()
    agents_awaiting_results = agents_awaiting_results + agents_result_expected_today*GOT_RESULT_VAR
    
    #agents_awaiting_results[agents_result_expected_today] = 0 # these agents are no longer waiting for result

    not_susceptible = (current_stages > SUSCEPTIBLE_VAR).long()
    not_recovered = (current_stages < RECOVERED_VAR).long()
    infected_exposed_agents = logical_and(not_susceptible, not_recovered)

    positive_result_candidates = logical_and(infected_exposed_agents, agents_result_expected_today)
    negative_result_candidates = logical_not(positive_result_candidates)

    positive_test_result = discrete_sample(sample_prob=true_positive_prob, size=positive_result_candidates.sum(), device='cpu')
    negative_test_result = discrete_sample(sample_prob=false_positive_prob, size=negative_result_candidates.sum(), device='cpu')

    tested_positive_today = positive_test_result
    # agents_test_results[negative_result_candidates] = negative_test_result
    # agents_test_results[positive_result_candidates] = positive_test_result

    return tested_positive_today

In [5]:
t = 3

current_stages = torch.tensor([3, 0, 2, 1, 2], dtype=torch.float32, requires_grad=True)
is_quarantined = torch.tensor([0, 0, 1, 0, 0])
agents_awaiting_test_results = torch.tensor([0, 0, 0, 0, 0], dtype=torch.float32)
test_compliance_prob = torch.tensor(0.8, requires_grad=True)
days_to_result = 4
agents_result_dates = torch.ones((num_agents))*INFINITY_TIME

new_agents_awaiting_test_results, new_agents_result_dates = get_tested(t, agents_result_dates, current_stages, is_quarantined, agents_awaiting_test_results, days_to_result, test_compliance_prob)
print(new_agents_awaiting_test_results)
new_agents_awaiting_test_results.sum().backward()
print(test_compliance_prob.grad)

test eligible and enrolled:  tensor([0., 0., 0., 1., 1.]) tensor([0., 0., 0., 0., 1.], grad_fn=<AddBackward0>)
tensor([0., 0., 0., 0., 1.], grad_fn=<AddBackward0>)
tensor(2.8196)


In [7]:
true_positive_prob = torch.tensor(0.7, requires_grad=True)
false_positive_prob = torch.tensor(0.3, requires_grad=True)

tested_positive_today = get_test_result(t, new_agents_awaiting_test_results, new_agents_result_dates, current_stages, true_positive_prob, false_positive_prob)
print(tested_positive_today)
(tested_positive_today + 3).sum().backward()
print(true_positive_prob.grad)

tensor([], grad_fn=<SelectBackward0>)
tensor(0.)


In [36]:
# covid-19 testing delays
result_delays = torch.tensor([2, 3, 4])
dist = torch.distributions.Categorical(result_delays)
dist.sample()

import numpy as np
elements = [2, 3, 4]
probabilities = [0.2, 0.5, 0.3]
np.random.choice(elements, 1, p=probabilities)

array([4])

In [ ]:
def random_choice(sample_tensor, device='cpu'):
    torch.randperm(sample_tensor, device=device)[0]

result_delays = torch.tensor([2, 3, 4])
delay_date = result_delays

Tasks to do for testing experiment:
1. sample delay time steps from a categorical distribution
2. what variable will to track positive tests to start quarantine?